In [1]:
# !matplotlib notebook
import matplotlib.pyplot as plt
from skimage import data
from skimage.exposure import histogram
import tifffile
import os
import sys
from skimage import data, feature, exposure
import numpy as np

path = "E:\\Sudipta\\Arpan"
file_name = "send-1.tif"

In [59]:
def scale_to(x,  dtype):
    r = np.max(x) - np.min(x)
    try:
        t_max = np.iinfo(np.dtype(dtype)).max
    except:
        t_max = np.finfo(np.dtype(dtype)).max
    
    # assert(math.isclose(0,r, abs_tol=np.finfo(float).eps) == False)
    x_s =  ((x - np.min(x)) / r) * t_max
    return x_s.astype(dtype)

def scale_to_float(x,  dtype_in):
#     r = np.max(x) - np.min(x)
    try:
        i_min = np.iinfo(np.dtype(dtype_in)).min
        i_max = np.iinfo(np.dtype(dtype_in)).max
    except:
        i_min = np.finfo(np.dtype(dtype_in)).min
        i_max = np.finfo(np.dtype(dtype_in)).max
    
    r = i_max -i_min
    return (x - i_min)/r
    
    # assert(math.isclose(0,r, abs_tol=np.finfo(float).eps) == False)
#     x_s =  ((x - np.min(x)) / r) * t_max
#     return x_s.astype(dtype)

def subtract(a, b, dtype='uint16'):
    try:
        t_min = np.iinfo(np.dtype(dtype)).min
        t_max = np.iinfo(np.dtype(dtype)).max
    except:
        t_min = np.finfo(np.dtype(dtype)).min
        t_max = np.finfo(np.dtype(dtype)).max        
    return np.clip(a.astype('int32') - b, t_min, t_max).astype(a.dtype)

def top_hat(image, radius=50, light_bg=False):
        from skimage.morphology import white_tophat, black_tophat, disk
        str_el = disk(radius) #you can also use 'ball' here to get a slightly smoother result at the cost of increased computing time
        if light_bg:
            return  scale_to(black_tophat(image, str_el), image.dtype)
        else:
            return  scale_to(white_tophat(image, str_el), image.dtype)

def thresholdV2(image, param=10):
    from skimage.filters import threshold_otsu
    dtype = image.dtype
    thresh = threshold_otsu(image)
    
    return image > thresh

def threshold(image, param=10):
    from skimage.filters import threshold_otsu
    dtype = image.dtype
    thresh = threshold_otsu(image)
    
    return thresh, subtract(image, thresh, image.dtype)

def dspeckle(image, param=10):
    from skimage.filters import median
    dtype = image.dtype
    return median(image)
    

def gaussian_filter(image, param=10):
    from skimage.filters import gaussian
    return scale_to(gaussian(image), image.dtype)


def plot_multi(images):
    fig, ax = plt.subplots(ncols=len(images), figsize=(20, 8))
    for i, image in enumerate(images.items()):
        ax[i].set_title(str(image[0]))
        ax[i].imshow(image[1], clim=[np.min(image[1]), np.max(image[1])], cmap='gray')
    plt.show()

def save_images(images, pre_fix="img"):
    os.makedirs(os.path.join(path,pre_fix), exist_ok=True)
    for key, img in images.items():
        p = os.path.join(path,pre_fix, f"{key}_image.tif")
        print()
        tifffile.imwrite(p, img)


In [4]:
from skimage import data
import napari
viewer = napari.Viewer()

In [16]:
data = tifffile.imread(os.path.join(path, "send-1.tif"))
image = data[157]
# viewer.add_image(image, name="Input")

In [17]:
_,threshold_img = threshold(image)
# viewer.add_image(threshold_img, name="threshold_img")

In [18]:
dspeckle_img = dspeckle(threshold_img)
# viewer.add_image(dspeckle_img, name="dspeckle_img")

In [19]:
gaussian_image = gaussian_filter(dspeckle_img)
# viewer.add_image(gaussian_image, name="gaussian_image")

In [20]:
_,gaussian_threshold_img = threshold(gaussian_image)
# viewer.add_image(gaussian_threshold_img, name="gaussian_threshold_img")

In [27]:
top_hat_img = top_hat(gaussian_threshold_img)
# viewer.add_image(top_hat_img, name="top_hat_img")
_, top_hat_thres_img = threshold(top_hat_img)
viewer.add_image(top_hat_thres_img, name="top_hat_thres_img")


<Image layer 'top_hat_thres_img' at 0x23c30d57f70>

In [65]:
# equalize_hist_img = exposure.equalize_hist(top_hat_thres_img)
# viewer.add_image(equalize_hist_img, name="equalize_hist_img")
from skimage.filters import median
thrr = median(top_hat_thres_img)
viewer.add_image(thrr, name="median")
# print(np.min(thrr))
# print(np.max(thrr))
# plt.imshow(thrr)

med = np.median(np.unique(thrr.flatten()))
print(med)
med = scale_to_float(med, top_hat_thres_img.dtype)
print(med)
img_p = feature.blob_log(thrr,  max_sigma =100, threshold=0.1)
print(img_p)
viewer.add_points(img_p[:, :2], size=(2,2), face_color='red', out_of_slice_display=True)


4658.0
0.07107652399481193
[[110.  67.   1.]
 [103. 136.   1.]
 [ 43. 128.   1.]
 [ 68. 125.   1.]
 [ 59.  20.   1.]]


<Points layer 'Points [6]' at 0x23c2b8feac0>

In [5]:
data = tifffile.imread(os.path.join(path, "send-1.tif"))
image = data[0]
def get_point2d(image):
    _,threshold_img = threshold(image)
    dspeckle_img = dspeckle(threshold_img)
    gaussian_image = gaussian_filter(dspeckle_img)
    _,gaussian_threshold_img = threshold(gaussian_image)
    top_hat_img = top_hat(gaussian_threshold_img)
    _, top_hat_thres_img = threshold(top_hat_img)
    img_p = feature.blob_log(top_hat_thres_img, overlap=0.9, threshold=0.1)
    return img_p[:, :2]

viewer.add_image(data, name="Main")
points = []
for i in range(data.shape[0]):
    print(i)
    _points = get_point2d(data[i])
    for j in range(_points.shape[0]):
        _p = list(_points[j])
        _p.insert(0, i)
        points.append(_p)

# print(points)
viewer.add_points(points, size=2, face_color='red', out_of_slice_display=True)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

<Points layer 'Points' at 0x23c073daf70>

In [6]:
print(points)

[[0, 86.0, 125.0], [0, 99.0, 51.0], [0, 56.0, 65.0], [0, 97.0, 59.0], [0, 90.0, 78.0], [0, 74.0, 81.0], [0, 90.0, 95.0], [0, 91.0, 113.0], [0, 54.0, 93.0], [0, 52.0, 78.0], [0, 45.0, 76.0], [0, 62.0, 68.0], [0, 61.0, 29.0], [0, 137.0, 47.0], [0, 64.0, 52.0], [0, 109.0, 45.0], [0, 89.0, 48.0], [0, 127.0, 75.0], [0, 64.0, 38.0], [0, 101.0, 122.0], [0, 55.0, 41.0], [0, 101.0, 84.0], [0, 101.0, 80.0], [0, 53.0, 57.0], [0, 110.0, 71.0], [0, 81.0, 133.0], [0, 104.0, 103.0], [0, 98.0, 28.0], [0, 56.0, 76.0], [0, 78.0, 148.0], [0, 64.0, 84.0], [1, 90.0, 95.0], [1, 56.0, 66.0], [1, 74.0, 82.0], [1, 64.0, 38.0], [1, 86.0, 126.0], [1, 91.0, 113.0], [1, 61.0, 29.0], [1, 89.0, 109.0], [1, 99.0, 51.0], [1, 51.0, 78.0], [1, 89.0, 47.0], [1, 97.0, 59.0], [2, 90.0, 95.0], [2, 56.0, 66.0], [2, 99.0, 51.0], [2, 74.0, 82.0], [2, 64.0, 38.0], [2, 89.0, 109.0], [2, 61.0, 29.0], [2, 127.0, 74.0], [2, 97.0, 60.0], [2, 91.0, 113.0], [2, 89.0, 48.0], [2, 137.0, 47.0], [2, 86.0, 84.0], [2, 102.0, 100.0], [2, 65.

In [7]:
print(data.shape)

(1248, 138, 181)


In [8]:
os.makedirs(os.path.join(path,"point_mask"), exist_ok=True)

In [15]:
shape = data.shape[1:3]
mask = np.zeros(data.shape, dtype='uint16')
for i in points:
    mask[int(i[0]), int(i[1]), int(i[2])] = 1


tifffile.imwrite(os.path.join(path,"point_mask", "points.tif"), mask)

In [26]:
from math import sqrt
from skimage import data
from skimage.feature import blob_dog, blob_log, blob_doh
from skimage.color import rgb2gray

import matplotlib.pyplot as plt


image = data.hubble_deep_field()[0:500, 0:500]
image_gray = rgb2gray(image)

blobs_log = blob_log(image_gray, max_sigma=30, num_sigma=10, threshold=.1)

print(blobs_log)
# Compute radii in the 3rd column.
blobs_log[:, 2] = blobs_log[:, 2] * sqrt(2)
print(blobs_log)

[[401.         123.          10.66666667]
 [199.         270.           4.22222222]
 [334.         308.           7.44444444]
 ...
 [108.         259.           1.        ]
 [375.         333.           1.        ]
 [ 46.         131.           1.        ]]
[[401.         123.          15.08494467]
 [199.         270.           5.97112393]
 [334.         308.          10.5280343 ]
 ...
 [108.         259.           1.41421356]
 [375.         333.           1.41421356]
 [ 46.         131.           1.41421356]]
